In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Heart_rate"
cohort = "GSE18583"

# Input paths
in_trait_dir = "../../input/GEO/Heart_rate"
in_cohort_dir = "../../input/GEO/Heart_rate/GSE18583"

# Output paths
out_data_file = "../../output/preprocess/Heart_rate/GSE18583.csv"
out_gene_data_file = "../../output/preprocess/Heart_rate/gene_data/GSE18583.csv"
out_clinical_data_file = "../../output/preprocess/Heart_rate/clinical_data/GSE18583.csv"
json_path = "../../output/preprocess/Heart_rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Baseline skeletal muscle gene expression"
!Series_summary	"Muscle biopsy samples were obtained from two groups of male subjects prior to endurance training. The samples were used to predict training responses."
!Series_summary	"Baseline gene expression involving 30 probe sets was able to classify subjects into high and low responders."
!Series_overall_design	"Resting skeletal muscle sample after an overnight fast."
Sample Characteristics Dictionary:
{0: ['gender: male'], 1: ['protocol: Resting skeletal muscle sample prior to endurance training'], 2: ['heart rate (bpm): 173', 'heart rate (bpm): 155', 'heart rate (bpm): 183', 'heart rate (bpm): 149', 'heart rate (bpm): 146', 'heart rate (bpm): 157', 'heart rate (bpm): 162', 'heart rate (bpm): 170', 'heart rate (bpm): 165', 'heart rate (bpm): 144', 'heart rate (bpm): 167', 'heart rate (bpm): 191', 'heart rate (bpm): 160', 'heart rate (bpm): 177', 'heart rate (bpm): 174', 'heart rate (bpm): 190', 'hea

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information which mentions "skeletal muscle gene expression" 
# and "30 probe sets", this dataset likely contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Heart rate data is available in row 2
trait_row = 2
# Age is not available in the sample characteristics
age_row = None
# Gender is available in row 0, but it's constant (all male)
gender_row = None  # Setting to None because it's constant (all male)

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert heart rate values to continuous numeric values."""
    if pd.isna(value):
        return None
    
    # Extract the numeric value after the colon
    try:
        # Extract from format like "heart rate (bpm): 173"
        heart_rate = float(value.split(': ')[1])
        return heart_rate
    except (IndexError, ValueError):
        return None

def convert_age(value):
    """Placeholder function for age conversion."""
    return None  # Age data not available

def convert_gender(value):
    """Placeholder function for gender conversion."""
    return None  # Gender data not available (all male)

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Conduct initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data (assuming it's available in a specific format)
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    # If clinical data is available in another format or location, adjust accordingly
    try:
        clinical_data = pd.read_csv(clinical_data_path)
    except FileNotFoundError:
        # If clinical_data.csv doesn't exist, try to find another file that might contain clinical data
        clinical_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.csv') and 'clinical' in f.lower()]
        if clinical_files:
            clinical_data = pd.read_csv(os.path.join(in_cohort_dir, clinical_files[0]))
        else:
            # If no clinical data file is found, create one from the sample characteristics dictionary
            sample_chars = {0: ['gender: male'], 
                            1: ['protocol: Resting skeletal muscle sample prior to endurance training'], 
                            2: ['heart rate (bpm): 173', 'heart rate (bpm): 155', 'heart rate (bpm): 183', 'heart rate (bpm): 149', 
                                'heart rate (bpm): 146', 'heart rate (bpm): 157', 'heart rate (bpm): 162', 'heart rate (bpm): 170', 
                                'heart rate (bpm): 165', 'heart rate (bpm): 144', 'heart rate (bpm): 167', 'heart rate (bpm): 191', 
                                'heart rate (bpm): 160', 'heart rate (bpm): 177', 'heart rate (bpm): 174', 'heart rate (bpm): 190', 
                                'heart rate (bpm): 169', np.nan]}
            
            # Convert to DataFrame format (assuming the structure is consistent)
            rows = []
            for i in range(max(len(vals) if isinstance(vals, list) else 0 for vals in sample_chars.values())):
                row = {}
                for key, vals in sample_chars.items():
                    if i < len(vals):
                        row[key] = vals[i]
                    else:
                        row[key] = np.nan
                rows.append(row)
            
            clinical_data = pd.DataFrame(rows)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview: {0: [nan], 1: [nan], 2: [183.0]}
Clinical data saved to ../../output/preprocess/Heart_rate/clinical_data/GSE18583.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 73
Header line: "ID_REF"	"GSM462215"	"GSM462216"	"GSM462217"	"GSM462218"	"GSM462219"	"GSM462220"	"GSM462221"	"GSM462222"	"GSM462223"	"GSM462224"	"GSM462225"	"GSM462226"	"GSM462227"	"GSM462228"	"GSM462229"	"GSM462230"	"GSM462231"	"GSM462232"	"GSM462233"	"GSM462234"	"GSM462235"	"GSM462236"	"GSM462237"	"GSM462238"	"GSM462239"	"GSM462240"	"GSM462241"	"GSM462242"	"GSM462243"	"GSM462244"	"GSM462245"	"GSM462246"	"GSM462247"	"GSM462248"	"GSM462249"	"GSM462250"	"GSM462251"	"GSM462252"	"GSM462253"	"GSM462254"	"GSM462255"
First data line: "ENST00000000233_at"	315.42	450.67	465.51	876.96	377.22	550.95	499.31	918.83	341.56	367.81	541.69	446.74	466.85	342.56	328.26	414.94	482.4	400.64	420.95	521.42	721.28	303.8	441.64	551.9	431.16	719.8	471	434.85	518.32	475.11	468.78	608.02	218.24	384.56	603.23	553.91	387.35	481.86	527.56	214.81	537.14
Index(['ENST00000000233_at', 'ENST00000000412_at', 'ENST00000000442_at',
       'ENST00000001008_at', 'ENST00000002125_at', 'ENST0000000216

### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers in the gene expression data
# Identifiers like 'ENST00000000233_at' are Ensembl transcript IDs (ENST)
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They need to be mapped to gene symbols for easier interpretation and analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
try:
    # Let's first look for specific annotation information in the SOFT file
    annotation_info = []
    with gzip.open(soft_file, 'rt') as file:
        for line in file:
            if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                print(f"Platform title: {line.strip()}")
            if line.startswith('!Platform_annotation') or line.startswith('!platform_annotation'):
                print(f"Platform annotation: {line.strip()}")
            if 'ANNOTATION information' in line or 'annotation information' in line:
                annotation_info.append(line.strip())
                
    if annotation_info:
        print("\nFound annotation information:")
        for info in annotation_info[:5]:  # Limit to first 5 to avoid overwhelming output
            print(info)
            
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    
    # If the gene annotation is too limited, try a more specific approach
    if len(gene_annotation.columns) <= 2:
        print("\nInitial gene annotation retrieval returned limited columns. Trying alternative approach...")
        
        # Try to extract the platform section manually which contains gene annotation
        platform_data = []
        with gzip.open(soft_file, 'rt') as file:
            in_platform_section = False
            for line in file:
                if line.startswith('^PLATFORM'):
                    in_platform_section = True
                    continue
                if in_platform_section and line.startswith('!platform_table_begin'):
                    # Skip the !platform_table_begin line
                    # Next line should be the header
                    header = next(file).strip()
                    platform_data.append(header)
                    # Read until the end of the platform table
                    for table_line in file:
                        if table_line.startswith('!platform_table_end'):
                            break
                        platform_data.append(table_line.strip())
                    break
        
        # Convert platform data to DataFrame if we found it
        if platform_data:
            import io
            platform_text = '\n'.join(platform_data)
            gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                          low_memory=False, on_bad_lines='skip')
    
    # 2. Preview the gene annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    print("\nGene annotation preview:")
    print(preview_df(gene_annotation))
    
    # Since we only have Ensembl transcript IDs without gene symbols in the annotation,
    # we'll need to create a mapping structure using the SPOT_ID (Ensembl ID)
    # This will be used for mapping in subsequent steps
    print("\nPreparing mapping from Ensembl transcript IDs...")
    # Create a mapping DataFrame with ID and SPOT_ID (which is the Ensembl ID without _at)
    mapping_df = gene_annotation[['ID', 'SPOT_ID']].copy()
    print("Mapping structure preview:")
    print(preview_df(mapping_df))
    
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Platform title: !Platform_title = Affymetrix GeneChip Human Genome U133 Plus 2.0 Array [HGU133Plus2_Hs_ENST (v10) CDF]



Initial gene annotation retrieval returned limited columns. Trying alternative approach...

Gene annotation columns:
['ID', 'SPOT_ID']

Gene annotation preview:
{'ID': ['ENST00000000233_at', 'ENST00000000412_at', 'ENST00000000442_at', 'ENST00000001008_at', 'ENST00000002125_at'], 'SPOT_ID': ['ENST00000000233', 'ENST00000000412', 'ENST00000000442', 'ENST00000001008', 'ENST00000002125']}

Preparing mapping from Ensembl transcript IDs...
Mapping structure preview:
{'ID': ['ENST00000000233_at', 'ENST00000000412_at', 'ENST00000000442_at', 'ENST00000001008_at', 'ENST00000002125_at'], 'SPOT_ID': ['ENST00000000233', 'ENST00000000412', 'ENST00000000442', 'ENST00000001008', 'ENST00000002125']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the gene identifiers and annotation data
# From previous steps, we observed:
# - Gene expression data has identifiers like ENST00000000233_at
# - Gene annotation data has 'ID' and 'SPOT_ID' columns
#   - 'ID' column matches the gene expression data index (Ensembl transcript IDs with _at suffix)
#   - 'SPOT_ID' contains Ensembl transcript IDs without the _at suffix

# Create a mapping dataframe with probe IDs and their corresponding Ensembl IDs
gene_annotation = get_gene_annotation(soft_file)

# Create a mapping dataframe using the provided annotation
# We'll use the Ensembl IDs directly since we don't have gene symbols
mapping_df = gene_annotation[['ID', 'SPOT_ID']].copy()
mapping_df = mapping_df.rename(columns={'SPOT_ID': 'Gene'})

# Print a sample of the mapping dataframe
print("Gene mapping dataframe sample:")
print(mapping_df.head())

# Since the apply_gene_mapping function expects string values in Gene column that can be extracted
# with extract_human_gene_symbols, but we want to use the Ensembl IDs directly, 
# we'll modify our approach

# First, get the gene expression data
gene_data = get_genetic_data(matrix_file)

# Create a simplified version of apply_gene_mapping for our Ensembl IDs
# This will directly map probes to Ensembl IDs without symbol extraction
def apply_direct_mapping(expression_df, mapping_df):
    # Make sure mapping_df has the necessary columns
    mapping_df = mapping_df[mapping_df['ID'].isin(expression_df.index)].copy()
    # Set 1-to-1 mapping as we're directly using the Ensembl IDs
    mapping_df['num_genes'] = 1
    mapping_df.set_index('ID', inplace=True)
    
    # Merge and distribute expression values
    merged_df = mapping_df.join(expression_df)
    expr_cols = [col for col in merged_df.columns if col not in ['Gene', 'num_genes']]
    
    # Group by Gene (Ensembl ID) and sum values (simple 1:1 mapping)
    gene_expression_df = merged_df.groupby('Gene')[expr_cols].sum()
    
    return gene_expression_df

# Apply our direct mapping function
gene_data = apply_direct_mapping(gene_data, mapping_df)

# Print a sample of the resulting gene expression data
print("\nGene expression data sample (after mapping):")
print(gene_data.shape)
print(gene_data.head())

# Save the gene data to the specified output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping dataframe sample:
                   ID             Gene
0  ENST00000000233_at  ENST00000000233
1  ENST00000000412_at  ENST00000000412
2  ENST00000000442_at  ENST00000000442
3  ENST00000001008_at  ENST00000001008
4  ENST00000002125_at  ENST00000002125



Gene expression data sample (after mapping):
(225803, 41)
      GSM462215  GSM462216  GSM462217  GSM462218  GSM462219  GSM462220  \
Gene                                                                     
0.05       3.84     117.86       5.48       4.81       4.86       0.12   
0.06      10.69      24.27      39.88      29.90      77.64      48.53   
0.12       7.68     235.72      10.96       9.62       9.72       0.24   
0.13      27.54     193.82      22.92      47.62     109.90       0.39   
0.14      49.13      74.34     128.75      69.59      92.31     101.45   

      GSM462221  GSM462222  GSM462223  GSM462224  ...  GSM462246  GSM462247  \
Gene                                              ...                         
0.05       0.25     103.00      17.13       6.13  ...       0.25      65.76   
0.06     146.23      52.53      41.74       2.58  ...       2.55      92.85   
0.12       0.50     206.00      34.26      12.26  ...       0.50     131.52   
0.13      25.56     176.19 


Gene expression data saved to ../../output/preprocess/Heart_rate/gene_data/GSE18583.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the clinical data again to ensure we have the correct data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get a proper view of the sample characteristics
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# Extract heart rate data using the convert_trait function from Step 2
def convert_trait(value):
    """Convert heart rate value to continuous numeric type."""
    if pd.isna(value):
        return None
    try:
        # Extract the numerical value after the colon and "bpm:"
        parts = value.split(":")
        if len(parts) < 2:
            return None
        numeric_value = parts[1].strip()
        # Remove possible 'bpm' text and convert to float
        numeric_value = numeric_value.replace("bpm", "").strip()
        return float(numeric_value)
    except (ValueError, IndexError):
        return None

# Gender conversion function (defined in Step 2)
def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if pd.isna(value):
        return None
    try:
        gender = value.split(":")[1].strip().lower()
        if "male" in gender:
            return 1
        elif "female" in gender:
            return 0
        else:
            return None
    except (ValueError, IndexError):
        return None

# Extract clinical features based on the rows identified in Step 2
trait_row = 2  # Heart rate data is in row 2
gender_row = 0  # Gender data is in row 0
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Transpose the clinical data for easier processing
selected_clinical_df = selected_clinical_df.T
selected_clinical_df.index.name = 'Sample'

# Save clinical data to file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Keep using the original gene expression data since mapping failed
# The gene_data object from Step 6 already contains our gene expression data
print(f"Gene data shape: {gene_data.shape}")

# Save the gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 3. Link clinical and genetic data
# Make sure sample IDs match between clinical and genetic data
common_samples = list(set(selected_clinical_df.index) & set(gene_data.columns))
if not common_samples:
    print("Warning: No matching sample IDs between clinical and genetic data!")
    # Try to match based on order rather than IDs if needed
    selected_clinical_df.index = gene_data.columns[:len(selected_clinical_df)]
    common_samples = list(selected_clinical_df.index)

# Select only common samples from both datasets
clinical_subset = selected_clinical_df.loc[common_samples]
gene_subset = gene_data[common_samples]

# Now link the data
linked_data = pd.concat([clinical_subset, gene_subset.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print(f"Linked data columns preview: {list(linked_data.columns[:10])}")

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait and col != 'Gender']
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing} out of {len(gene_cols)}")
    
    if len(linked_data) > 0:
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing} out of {len(linked_data)}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from skeletal muscle samples before and after endurance training, with heart rate measurements."

is_gene_available = len(gene_data) > 0
is_trait_available = True  # We've confirmed trait data is available

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Clinical data saved to ../../output/preprocess/Heart_rate/clinical_data/GSE18583.csv
Gene data shape: (225803, 41)


Gene data saved to ../../output/preprocess/Heart_rate/gene_data/GSE18583.csv
Linked data shape: (41, 225805)
Linked data columns preview: ['Heart_rate', 'Gender', '0.05', '0.06', '0.12', '0.13', '0.14', '0.15', '0.16', '0.17']

Missing values before handling:
  Trait (Heart_rate) missing: 17 out of 41
  Genes with >20% missing: 0 out of 225803


  Samples with >5% missing genes: 0 out of 41


Data shape after handling missing values: (24, 225805)
Quartiles for 'Heart_rate':
  25%: 159.25
  50% (Median): 166.0
  75%: 174.75
Min: 144.0
Max: 191.0
The distribution of the feature 'Heart_rate' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 24 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Heart_rate/GSE18583.csv
